# Aligning Data to Darwin Core - Sampling Event with Measurement or Fact using Python
Matt Biddle

November 9, 2020

# General information about this notebook
This notebook was created for the IOOS DMAC Code Sprint Biological Data Session
The data in this notebook were created specifically as an example and meant solely to be
illustrative of the process for aligning data to the biological data standard - Darwin Core.
These data should not be considered actually occurrences of species and any measurements
are also contrived. This notebook is meant to provide a step by step process for taking
original data and aligning it to Darwin Core

This notebook is a python implementation of the R notebook [IOOS_DMAC_DataToDWC_Notebook_event.R](https://github.com/ioos/bio_data_guide/blob/master/Standardizing%20Marine%20Biological%20Data/datasets/example_script_with_fake_data/IOOS_DMAC_DataToDwC_Notebook_event.R)

In [1]:
import pandas as pd
import pyworms # pip install git+git://github.com/iobis/pyworms.git
import numpy as np
import uuid
import csv

Read in the raw data file.

In [2]:
url = "http://www.neracoos.org/erddap/tabledap/WBTS_CFIN_2004_2017.csv"
df = pd.read_csv(url, header=[0])

df.columns

Index(['Cruise_Identification_Tag', 'CRUISE_ID', 'Station_ID', 'latitude',
       'longitude', 'time', 'Cast', 'Net_Type', 'Mesh_Size', 'NET_DEPTH',
       'STATION_DEPTH', 'COMMENT', 'Plankton_Net_Area', 'Volume_Filtered',
       'Sample_Split', 'Sample_Dry_Weight', 'DW_G_M_2', 'Dilution_Factor',
       'TOTAL_DILFACTOR_CFIN', 'Order', 'Calanus_finmarchicus_N',
       'Calanus_finmarchicus_CI', 'Calanus_finmarchicus_CII',
       'Calanus_finmarchicus_CIII', 'Calanus_finmarchicus_CIV',
       'Calanus_finmarchicus_CV', 'Calanus_finmarchicus_F',
       'Calanus_finmarchicus_M'],
      dtype='object')

## Input columns:
1. Cruise_Identification_Tag
2. CRUISE_ID
3. Station_ID
4. latitude
5. longitude
6. time
7. Cast
8. Net_Type
9. Mesh_Size
10. NET_DEPTH
11. STATION_DEPTH
12. COMMENT
13. Plankton_Net_Area
14. Volume_Filtered
15. Sample_Split
16. Sample_Dry_Weight
17. DW_G_M_2
18. Dilution_Factor
19. TOTAL_DILFACTOR_CFIN
20. Order
21. Calanus_finmarchicus_N
22. Calanus_finmarchicus_CI
23. Calanus_finmarchicus_CII
24. Calanus_finmarchicus_CIII
25. Calanus_finmarchicus_CIV
26. Calanus_finmarchicus_CV
27. Calanus_finmarchicus_F
28. Calanus_finmarchicus_M

## Mappings:
**Event Table**  

| *Origin Term*             | *DwC_term(s)*                              |  *Notes*               |
|---------------------------|--------------------------------------------|------------------------|
| Cruise_Identification_Tag | eventID                                    | eventID                |
| CRUISE_ID                 | eventID                                    | contained in eventID   |
| Station_ID                | eventID                                    | contained in eventID   |
| cast                      | eventID                                    | contained in eventID   |
| latitude                  | decimalLatitude                            |                        |
| longitude                 | decimalLongitude                           |                        |
| STATION_DEPTH             | minimumDepthInMeters, maximumDepthInMeters |                        |
| time                      | eventDate                                  |                        |
|                           | geodeticDatum                              | added programatically  |
|                           | samplingProtocol                           | added manually         |

**Occurrence Table**

The `Calanus_finmarchius_*` readings are split into individual records, with the following fields added:

| *Origin Term*         | *DwC_term(s)*            | *Notes*                           |
|-----------------------|--------------------------|-----------------------------------|
| `Calanus_finmarchius_*` | individualCount        | original value under each column  |
|                       | scientificName           | derived from original column name |
|                       | occurrenceStatus         | added programatically             |
|                       | lifeStage                | derived from original column name |
|                       | sex                      | derived from original column name |
|                       | acceptedname             | programatic pyworms lookup        |
|                       | acceptedID               | programatic pyworms lookup        |
|                       | scientificNameID         | programatic pyworms lookup        |
|                       | kingdom                  | programatic pyworms lookup        |
|                       | phylum                   | programatic pyworms lookup        |
|                       | class                    | programatic pyworms lookup        |
|                       | order                    | programatic pyworms lookup        |
|                       | family                   | programatic pyworms lookup        |
|                       | genus                    | programatic pyworms lookup        |
|                       | scientificNameAuthorship | programatic pyworms lookup        |
|                       | taxonRank                | programatic pyworms lookup        |
|                       | basisOfRecord            | added programatically             |

**Measurement or Fact Table**

Each entry in this table has the following fields:

1. measurementType
2. measurementTypeID
3. measurementValue
4. measurementUnit
5. measurementUnitID
6. measurementAccuracy
7. measurementDeterminedDate
8. measurementMethod
9. measurementRemark

This table shows the mapping from the origin term to the BODC NERC vocabulary term:

| *Origin Term*        | *BODC NERC vocabulary/measurementTypeID*                              | *URI*                                                                |
|----------------------|-----------------------------------------------------------------------|----------------------------------------------------------------------|
| Net_Type             | plankton net                                                          | [22](http://vocab.nerc.ac.uk/collection/L05/current/22/)             |
| Mesh_Size            | Sampling net mesh size                                                | [Q0100015](http://vocab.nerc.ac.uk/collection/Q01/current/Q0100015/) |
| NET_DEPTH            | Depth (spatial coordinate) of sampling event start                    | [DXPHPRST](http://vocab.nerc.ac.uk/collection/P01/current/DXPHPRST/) |
| COMMENT              | N/A (mapped to measurementRemark field above)                         | N/A                                                                  |
| Plankton_Net_Area    | Sampling device aperture surface area                                 | [Q0100017](http://vocab.nerc.ac.uk/collection/Q01/current/Q0100017/) |
| Volume_Filtered      | Volume                                                                | [VOL](http://vocab.nerc.ac.uk/collection/P25/current/VOL/)           |
| Sample_Split         | N/A (information added to measurementRemark field above)              | N/A                                                                  |
| Sample_Dry_Weight    | Dry weight biomass                                                    | [ODRYBM01](http://vocab.nerc.ac.uk/collection/P01/current/ODRYBM01/) |
| DW_G_M_2             | Dry weight biomass                                                    | [ODRYBM01](http://vocab.nerc.ac.uk/collection/P01/current/ODRYBM01)  |
| Dilution_Factor      | ???                                                                   | ???                                                                  |
| TOTAL_DILFACTOR_CFIN | ???                                                                   | ???                                                                  |

First we need to to decide if we will provide an occurrence only version of the data or
a sampling event with measurement or facts version of the data. Occurrence only is easier
to create. It's only one file to produce. However, several pieces of information will be
left out if we choose that option. If we choose to do sampling event with measurement or
fact we'll be able to capture all of the data in the file creating a lossless version.
Here we decide to use the sampling event option to include as much information as we can.

First let's create the eventID and occurrenceID in the original file so that information
can be reused for all necessary files down the line. Luckily, our data already has an appropriate eventID in the `Cruise_Identification_Tag` field, so we'll use that. 

In [3]:
df['eventID'] = df['Cruise_Identification_Tag']
df['occurrenceID'] = uuid.uuid4()

# Event file

We will need to create three separate files to comply with the sampling event format.
We'll start with the event file but we only need to include the columns that are relevant
to the event file.

In [4]:
event = df[['time', 'latitude', 'longitude', 'NET_DEPTH', 'STATION_DEPTH', 'eventID']].copy()

Next we need to rename any columns of data that match directly to Darwin Core. We know
this based on our crosswalk spreadsheet CrosswalkToDarwinCore.csv

In [5]:

event['eventDate'] = event['time']
event['decimalLatitude'] = event['latitude']
event['decimalLongitude'] = event['longitude']
event['minimumDepthInMeters'] = event['NET_DEPTH']
event['maximumDepthInMeters'] = event['NET_DEPTH']

Let's see how it looks:

In [6]:
event.head()

,time,latitude,longitude,NET_DEPTH,STATION_DEPTH,eventID,eventDate,decimalLatitude,decimalLongitude,minimumDepthInMeters,maximumDepthInMeters
0,UTC,degrees_north,degrees_east,meters,meters,NaN,UTC,degrees_north,degrees_east,meters,meters
1,2004-12-06T00:00:00Z,42.8612,-69.8605,255,260,GC120604WBWB-72,2004-12-06T00:00:00Z,42.8612,-69.8605,255,255
2,2005-01-30T00:00:00Z,42.8646,-69.8632,250,255,GC013005WBWB-71,2005-01-30T00:00:00Z,42.8646,-69.8632,250,250
3,2005-01-30T00:00:00Z,42.8646,-69.8632,250,255,GC013005WBWB-72,2005-01-30T00:00:00Z,42.8646,-69.8632,250,250
4,2005-02-14T00:00:00Z,42.8646,-69.8632,240,245,GC021405WBWB-71,2005-02-14T00:00:00Z,42.8646,-69.8632,240,240


We will also have to add any missing required fields

In [7]:
# this is a guess
event['geodeticDatum'] = 'EPSG:4326 WGS84'
# this is found in the metadata
event['samplingProtocol'] = 'Mesh net cast'

Then we'll remove any columns that we no longer need to clean things up a bit.

In [8]:
event.drop(
    columns=['latitude', 'longitude', 'NET_DEPTH', 'time'],
    inplace=True)

We have too many repeating rows of information. We can pare this down using eventID which
is a unique identifier for each sampling event in the data- which is six, three transects
per site.

In [9]:
event.drop_duplicates(
    subset='eventID',
    inplace=True)

event.head(6)

,STATION_DEPTH,eventID,eventDate,decimalLatitude,decimalLongitude,minimumDepthInMeters,maximumDepthInMeters,geodeticDatum,samplingProtocol
0,meters,NaN,UTC,degrees_north,degrees_east,meters,meters,EPSG:4326 WGS84,Mesh net cast
1,260,GC120604WBWB-72,2004-12-06T00:00:00Z,42.8612,-69.8605,255,255,EPSG:4326 WGS84,Mesh net cast
2,255,GC013005WBWB-71,2005-01-30T00:00:00Z,42.8646,-69.8632,250,250,EPSG:4326 WGS84,Mesh net cast
3,255,GC013005WBWB-72,2005-01-30T00:00:00Z,42.8646,-69.8632,250,250,EPSG:4326 WGS84,Mesh net cast
4,245,GC021405WBWB-71,2005-02-14T00:00:00Z,42.8646,-69.8632,240,240,EPSG:4326 WGS84,Mesh net cast
5,245,GC021405WBWB-72,2005-02-14T00:00:00Z,42.8646,-69.8632,240,240,EPSG:4326 WGS84,Mesh net cast


Finally we write out the event file

In [10]:
event.to_csv(
    'results/WBTS_CFIN_2004_2017_event_frompy.csv',
    header=True,
    index=False,)

# Occurrence file
Next we need to create the occurrence file. We start by examining the structure (columns) of the source data. The goal here is to assess what kind of conversion (if any) will be necessary for Darwin Core alignment. 

In [11]:
df.head(10)

,Cruise_Identification_Tag,CRUISE_ID,Station_ID,latitude,longitude,time,Cast,Net_Type,Mesh_Size,NET_DEPTH,...,Calanus_finmarchicus_N,Calanus_finmarchicus_CI,Calanus_finmarchicus_CII,Calanus_finmarchicus_CIII,Calanus_finmarchicus_CIV,Calanus_finmarchicus_CV,Calanus_finmarchicus_F,Calanus_finmarchicus_M,eventID,occurrenceID
0,NaN,NaN,NaN,degrees_north,degrees_east,UTC,NaN,NaN,microns,meters,...,Number of all nauplius stages per m2,Number of stage CI per m2,Number of stage CII per m2,Number of stage CIIII per m2,Number of stage CIV per m2,Number of stage CV per m2,Number of adult females per m2,Number of adult males per m2,NaN,eaad3acb-e6ce-4d85-a613-cc084697c28a
1,GC120604WBWB-72,GC120604WB,WB-7,42.8612,-69.8605,2004-12-06T00:00:00Z,2,.75DRing,200,255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GC120604WBWB-72,eaad3acb-e6ce-4d85-a613-cc084697c28a
2,GC013005WBWB-71,GC013005WB,WB-7,42.8646,-69.8632,2005-01-30T00:00:00Z,1,.75DRing,200,250,...,544,544,0,0,0,6251,4892,1359,GC013005WBWB-71,eaad3acb-e6ce-4d85-a613-cc084697c28a
3,GC013005WBWB-72,GC013005WB,WB-7,42.8646,-69.8632,2005-01-30T00:00:00Z,2,.75DRing,200,250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GC013005WBWB-72,eaad3acb-e6ce-4d85-a613-cc084697c28a
4,GC021405WBWB-71,GC021405WB,WB-7,42.8646,-69.8632,2005-02-14T00:00:00Z,1,.75DRing,200,240,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GC021405WBWB-71,eaad3acb-e6ce-4d85-a613-cc084697c28a
5,GC021405WBWB-72,GC021405WB,WB-7,42.8646,-69.8632,2005-02-14T00:00:00Z,2,.75DRing,200,240,...,4892,1902,544,0,2718,6794,11958,2989,GC021405WBWB-72,eaad3acb-e6ce-4d85-a613-cc084697c28a
6,GC031805WBWB-71,GC031805WB,WB-7,42.8626,-69.8621,2005-03-18T00:00:00Z,1,.75DRing,200,244,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GC031805WBWB-71,eaad3acb-e6ce-4d85-a613-cc084697c28a
7,GC031805WBWB-72,GC031805WB,WB-7,42.8626,-69.8621,2005-03-18T00:00:00Z,2,.75DRing,200,244,...,1223,408,272,136,136,5028,6658,1766,GC031805WBWB-72,eaad3acb-e6ce-4d85-a613-cc084697c28a
8,GC040605WBWB-71,GC040605WB,WB-7,42.8626,-69.8621,2005-04-06T00:00:00Z,1,.75DRing,200,258,...,6522,2174,272,408,272,1087,8153,136,GC040605WBWB-71,eaad3acb-e6ce-4d85-a613-cc084697c28a
9,GC040605WBWB-72,GC040605WB,WB-7,42.8626,-69.8621,2005-04-06T00:00:00Z,2,.75DRing,200,258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GC040605WBWB-72,eaad3acb-e6ce-4d85-a613-cc084697c28a


In this case, the `Calanus_finmarchicus` columns need to be converted into a more suitable format. We need to iterate through the existing data row by row - the goal is to create five new columns: `scientificName`, `lifeStage`, `sex`, `occuranceStatus`, & `individualCount`.

We start by isolating the records that have valid data. We define the columns we want to check against as `target_data_columns`, and then create a new dataframe `calanus_records` by retaining only records where at least one of the columns has a value of NOT `0` AND NOT `NaN`.

We also drop the second row, which contains unit information to avoid confusing the parser. 

In [12]:
target_data_columns = ['Calanus_finmarchicus_N',
                       'Calanus_finmarchicus_CI',
                       'Calanus_finmarchicus_CII',
                       'Calanus_finmarchicus_CIII',
                       'Calanus_finmarchicus_CIV',
                       'Calanus_finmarchicus_CV',
                       'Calanus_finmarchicus_F',
                       'Calanus_finmarchicus_M']

# drop units row from calanus records

calanus_records = df.iloc[1:, :]

The challenge is that, in its current form, each row actually represents between 0 and 8 discrete occurances. This isn't suitable for Darwin Core, so we need to read each row, and then split its data into new records, each representing an occurance event. This is a little tricky, so we'll create a helper method `enumerate_row` which takes a row (a `pandas.Series` object in practice) and makes the appropriate transformations. 

In [13]:
def enumerate_row(row, field):
    # expands rows which contain multiple observations into discrete records
    row_data = row[1]
    calanus_count = row_data[field]

    # convert to dict so we can mutate
    enumerated_row = row_data.to_dict()

    split_species = field.rsplit('_', 1)
    scientific_name = split_species[0].replace('_', ' ')
    life_stage = split_species[1]

    # add count of specified species as a new column
    enumerated_row['individualCount'] = calanus_count
    enumerated_row['scientificName'] = scientific_name
    
    enumerated_row['occurrenceStatus'] = 'present' if pd.to_numeric(calanus_count) > 0 and calanus_count != 'NaN' else 'absent'

    life_stage = field.rsplit('_', 1)[1]
    if life_stage == 'N':
        life_stage = 'Nauplius'
    enumerated_row['lifeStage'] = life_stage if life_stage != 'F' and life_stage != 'M' else 'adult'

    # this is consistent across records
    enumerated_row['basisOfRecord'] = 'HumanObservation'

    if life_stage == 'F':
        enumerated_row['sex'] = 'female'
    elif life_stage == 'M':
        enumerated_row['sex'] = 'male'
    else:
        enumerated_row['sex'] = 'NA'

    return enumerated_row

The next step is to loop through the target data. The top-level control variable is the list of the columns we wish to enumerate, so we will look for each target column in each row of the dataset. 

*note*: This operation could easily become costly depending on the number of rows and target columns

In [14]:
enumerated_rows = []
# loop through target column list
for field in target_data_columns:

    # now enumerate each input row, extracting the values
    for row in calanus_records.iterrows():

        flipped_row = enumerate_row(row, field)

        # delete other calanus records from flipped row
        for k in target_data_columns:
            flipped_row.pop(k, None)

        enumerated_rows.append(flipped_row)

A little bit of clean up:

In [15]:
# now convert the list of dicts into a dataframe
output_frame = pd.DataFrame.from_dict(enumerated_rows)

# sort by time, ascending
output_frame.sort_values(by='time', ascending=True, inplace=True)

Now our data should be in a more suitable fromat, so we can proceed. 

We start by creating a new occurrence data frame with the relevant fields.

In [16]:
occurrence = output_frame[['scientificName', 'eventID', 'occurrenceID', 'individualCount', 'occurrenceStatus', 'lifeStage', 'sex']].copy()

## Taxonomic Name Matching
A requirement for OBIS is that all scientific names match to the World Register of
Marine Species (WoRMS) and a scientificNameID is included. A scientificNameID looks
like this "urn:lsid:marinespecies.org:taxname:275730" with the last digits after
the colon being the WoRMS aphia ID. We'll need to go out to WoRMS to grab this
information.

Create a lookup table of unique scientific names

In [17]:
lut_worms = pd.DataFrame(
    columns=['scientificName'],
    data=occurrence['scientificName'].unique())

Add the columns that we can grab information from WoRMS including the required scientificNameID.

In [18]:
headers = ['acceptedname', 'acceptedID', 'scientificNameID', 'kingdom', 'phylum',
           'class', 'order', 'family', 'genus', 'scientificNameAuthorship', 'taxonRank']

for head in headers:
    lut_worms[head] = ''

Taxonomic lookup using the library [pyworms](https://github.com/iobis/pyworms)

In [19]:
for index, row in lut_worms.iterrows():
    print('Searching for scientific name = %s' % row['scientificName'])
    resp = pyworms.aphiaRecordsByMatchNames(row['scientificName'])[0][0]
    lut_worms.loc[index, 'acceptedname'] = resp['valid_name']
    lut_worms.loc[index, 'acceptedID'] = resp['valid_AphiaID']
    lut_worms.loc[index, 'scientificNameID'] = resp['lsid']
    lut_worms.loc[index, 'kingdom'] = resp['kingdom']
    lut_worms.loc[index, 'phylum'] = resp['phylum']
    lut_worms.loc[index, 'class'] = resp['class']
    lut_worms.loc[index, 'order'] = resp['order']
    lut_worms.loc[index, 'family'] = resp['family']
    lut_worms.loc[index, 'genus'] = resp['genus']
    lut_worms.loc[index, 'scientificNameAuthorship'] = resp['authority']
    lut_worms.loc[index, 'taxonRank'] = resp['rank']

Searching for scientific name = Calanus finmarchicus


Merge the lookup table of unique scientific names back with the occurrence data.

In [20]:
occurrence = pd.merge(occurrence, lut_worms, how='left', on='scientificName')

Quick look at what we have before we write out the file

In [21]:
occurrence.head()

,scientificName,eventID,occurrenceID,individualCount,occurrenceStatus,lifeStage,sex,acceptedname,acceptedID,scientificNameID,kingdom,phylum,class,order,family,genus,scientificNameAuthorship,taxonRank
0,Calanus finmarchicus,GC120604WBWB-72,eaad3acb-e6ce-4d85-a613-cc084697c28a,NaN,absent,Nauplius,NA,Calanus finmarchicus,104464,urn:lsid:marinespecies.org:taxname:104464,Animalia,Arthropoda,Hexanauplia,Calanoida,Calanidae,Calanus,"(Gunnerus, 1770)",Species
1,Calanus finmarchicus,GC120604WBWB-72,eaad3acb-e6ce-4d85-a613-cc084697c28a,NaN,absent,CIV,NA,Calanus finmarchicus,104464,urn:lsid:marinespecies.org:taxname:104464,Animalia,Arthropoda,Hexanauplia,Calanoida,Calanidae,Calanus,"(Gunnerus, 1770)",Species
2,Calanus finmarchicus,GC120604WBWB-72,eaad3acb-e6ce-4d85-a613-cc084697c28a,NaN,absent,CV,NA,Calanus finmarchicus,104464,urn:lsid:marinespecies.org:taxname:104464,Animalia,Arthropoda,Hexanauplia,Calanoida,Calanidae,Calanus,"(Gunnerus, 1770)",Species
3,Calanus finmarchicus,GC120604WBWB-72,eaad3acb-e6ce-4d85-a613-cc084697c28a,NaN,absent,adult,male,Calanus finmarchicus,104464,urn:lsid:marinespecies.org:taxname:104464,Animalia,Arthropoda,Hexanauplia,Calanoida,Calanidae,Calanus,"(Gunnerus, 1770)",Species
4,Calanus finmarchicus,GC120604WBWB-72,eaad3acb-e6ce-4d85-a613-cc084697c28a,NaN,absent,CII,NA,Calanus finmarchicus,104464,urn:lsid:marinespecies.org:taxname:104464,Animalia,Arthropoda,Hexanauplia,Calanoida,Calanidae,Calanus,"(Gunnerus, 1770)",Species


Write out the file.

In [22]:
# sort the columns on scientificName
occurrence.sort_values('scientificName', inplace=True)
# reorganize column order to be consistent with R example:
columns = ["scientificName","eventID","occurrenceID","occurrenceStatus","acceptedname","acceptedID",
           "scientificNameID","kingdom","phylum","class","order","family","genus","scientificNameAuthorship",
           "taxonRank"]

occurrence.to_csv(
    "results/WBTS_CFIN_2004_2017_occurrence_frompy.csv",
    header=True,
    index=False,
    quoting=csv.QUOTE_ALL,
    columns=columns)

 All done with occurrence!

# Measurement Or Fact
The last file we need to create is the measurement or fact file. For this we need to
combine all of the measurements or facts that we want to include making sure to include
IDs from the BODC NERC vocabulary where possible.

Now we'll manually map the remaining variables to the BODC NERC vocabulary when possible. For now we're mapping the following metadata for each field:

1. uri -> URL of the concept page on the NERC VOcabulary Server (NVS)
2. unit
3. unitId -> URL of the unit ID page on NVS
4. accuracy 
5. type -> measurement type 

In [23]:
df.columns

vocab_url_prefix = 'http://vocab.nerc.ac.uk/collection/'

column_mappings = {
    'Net_Type': {'uri': 'L05/current/22/', 'unit': '', 'unitID': '', 'accuracy': '', 'type': ''},
    'Mesh_Size': {'uri': 'Q01/current/Q0100015/', 'unit': 'microns', 'unitID': 'P06/current/UMIC/', 'accuracy': '', 'type': ''},
    'NET_DEPTH': {'uri': 'P01/current/DXPHPRST/', 'unit': 'm', 'unitID': 'P06/current/UPAA/', 'accuracy': '', 'type': ''},
    'Plankton_Net_Area': {'uri': 'Q01/current/Q0100017/', 'unit': 'm2', 'unitID': 'P06/current/UPAA/', 'accuracy': '', 'type': ''},
    'Volume_Filtered': {'uri': 'P25/current/VOL/', 'unit': 'm3', 'unitID': 'P06/current/UPAA/', 'accuracy': '', 'type': ''},
    'Sample_Dry_Weight': {'uri': 'P01/current/ODRYBM01/', 'unit': 'g', 'unitID': 'P06/current/UGRM/', 'accuracy': '', 'type': ''},
    'DW_G_M_2': {'uri': 'P01/current/ODRYBM01/', 'unit': 'g/m2', 'unitID': 'P06/current/UGMS/', 'accuracy': '', 'type': ''},
    'Dilution_Factor': {'uri': '', 'unit': 'ml', 'unitID': 'P06/current/VVML/', 'accuracy': '', 'type': ''},
    'TOTAL_DILFACTOR_CFIN': {'uri': '', 'unit': 'ml', 'unitID': 'P06/current/VVML/', 'accuracy': '', 'type': ''},
}

No we'll loop through the mapping list and transform as needed.

In [24]:
frames_to_concat = []

for current_field in column_mappings:

    current_mapping = column_mappings.get(current_field)
    
    current_df = df[['eventID', current_field, 'time', 'COMMENT', 'Sample_Split']].copy()

    #drop units row here
    current_df = current_df.iloc[1:, :]

    current_df['occurrenceID'] = ''
    current_df['measurementType'] = current_mapping.get('type')
    current_df['measurementTypeID'] = vocab_url_prefix + current_mapping.get('uri')
    current_df['measurementValue'] = current_df[current_field]
    current_df['measurementUnit'] = current_mapping.get('unit')
    current_df['measurementUnitID'] = vocab_url_prefix + current_mapping.get('unitID') if current_mapping.get('unitID') else ''
    current_df['measurementAccuracy'] = current_mapping.get('accuracy')
    current_df['measurementDeterminedDate'] = current_df['time']
    current_df.drop(
        columns=[current_field, 'time'],
        inplace=True)
    
    frames_to_concat.append(current_df)


Concatenate all measurements or facts together.

In [25]:
measurementorfact = pd.concat(frames_to_concat)

Let's check to see what it looks like

In [26]:
measurementorfact.head(50)

,eventID,COMMENT,Sample_Split,occurrenceID,measurementType,measurementTypeID,measurementValue,measurementUnit,measurementUnitID,measurementAccuracy,measurementDeterminedDate
1,GC120604WBWB-72,NaN,0.5,,,http://vocab.nerc.ac.uk/collection/L05/current...,.75DRing,,,,2004-12-06T00:00:00Z
2,GC013005WBWB-71,NaN,0.5,,,http://vocab.nerc.ac.uk/collection/L05/current...,.75DRing,,,,2005-01-30T00:00:00Z
3,GC013005WBWB-72,NaN,0.5,,,http://vocab.nerc.ac.uk/collection/L05/current...,.75DRing,,,,2005-01-30T00:00:00Z
4,GC021405WBWB-71,NaN,0.5,,,http://vocab.nerc.ac.uk/collection/L05/current...,.75DRing,,,,2005-02-14T00:00:00Z
5,GC021405WBWB-72,NaN,0.5,,,http://vocab.nerc.ac.uk/collection/L05/current...,.75DRing,,,,2005-02-14T00:00:00Z
6,GC031805WBWB-71,NaN,0.5,,,http://vocab.nerc.ac.uk/collection/L05/current...,.75DRing,,,,2005-03-18T00:00:00Z
7,GC031805WBWB-72,NaN,0.5,,,http://vocab.nerc.ac.uk/collection/L05/current...,.75DRing,,,,2005-03-18T00:00:00Z
8,GC040605WBWB-71,NaN,0.5,,,http://vocab.nerc.ac.uk/collection/L05/current...,.75DRing,,,,2005-04-06T00:00:00Z
9,GC040605WBWB-72,NaN,0.5,,,http://vocab.nerc.ac.uk/collection/L05/current...,.75DRing,,,,2005-04-06T00:00:00Z
10,GC051005WBWB-71,NaN,0.5,,,http://vocab.nerc.ac.uk/collection/L05/current...,.75DRing,,,,2005-05-10T00:00:00Z


Now we need to add in the remaining fields:

1. `measurementMethod`
2. `measurementRemark`

In [27]:
# this is a constant value as described in the metadata
measurementorfact['measurementMethod'] = 'Net used: 0.75 meter diameter single ring or a SEA-GEAR Model 9600 twin-ring, 200µm mesh'

# this is a constant value, PLUS anything in the 'COMMENT' field for a given occurrence
measurementorfact['measurementRemark'] = 'Note: no matching NERC vocabulary URI for sampling device. Comments: ' + df['COMMENT'].astype(str)

# drop COMMENT column as we don't need it
measurementorfact.drop(columns=['COMMENT'], inplace=True)

Write measurement or fact file

In [28]:
measurementorfact.to_csv('results/WBTS_CFIN_2004_2017_mof_frompy.csv',
                         index=False,
                         header=True)